In [1]:
class TInt:
    name = "TInt"

class TLong:
    name = "TLong"

class TDouble:
    name = "TDouble"

class TUInt:
    name = "TUInt"

class TULong:
    name = "TULong"
    
class TIdentifier:
    name = "TIdentifier"
    
class List:
    name = "List"

In [2]:
class AstNode:
    name = ""
    fields = []
    children = []

    def __init__(self, name, fields, children):
        self.name = name
        self.fields = fields
        self.children = children

In [3]:
def to_string(field_type, field_name):
    if field_type == "TIdentifier":
        return field_name
    else:
        return f"std::to_string({field_name})"
    
def print_node(node):
    print(f"        case AST_T::{node.name}_t: {{")
    print(f"            field(\"{node.name}\", \"\", ++t);")
    print(f"            {node.name}* p_node = static_cast<{node.name}*>(node);")
    
    for field in node.fields:
        if field[1][0] == "[":
            print(f"            field(\"List\", \"\", t+1);")
            print(f"            for(const auto& item: p_node->{field[1][1:]}) {{")
            print(f"                field(\"{field[0]().name}\", {to_string(field[0]().name, 'item')}, t+2);")
            print(f"            }}")
        else:
            print(f"            field(\"{field[0]().name}\", {to_string(field[0]().name, 'p_node->' + field[1])}, t+1);")
    
    for child in node.children:
        if child[0] == "[":
            print(f"            field(\"List\", \"\", t+1);")
            print(f"            for(const auto& item: p_node->{child[1:]}) {{")
            print(f"                print_ast(item.get(), t+1);")
            print(f"            }}")
        else:
            print(f"            print_ast(p_node->{child}.get(), t);")
    
    print(f"            break;")
    print(f"        }}")

In [4]:
# def print_base(name):
#     print(f"        case AST_T::{name}_t: {{")
#     print(f"            break;")
#     print(f"        }}")

In [5]:
# ast_bases = [
# #     "CExp",
# ]

In [6]:
ast_nodes = [
    AstNode("CAdd", 
            [], 
            []),
    AstNode("CConstInt", 
            [(TInt, "value")], 
            []),
    AstNode("CConstLong", 
            [(TLong, "value")], 
            []),
    AstNode("CConstant", 
            [], 
            ["constant"]),
    AstNode("CBinary", 
            [], 
            ["binary_op", "exp_left", "exp_right"]),
    AstNode("CExpression", 
            [], 
            ["exp"]),
    AstNode("CS",
            [],
            ["statement"]),
    AstNode("CB",
            [],
            ["[block_items"]),
    AstNode("CLabel", 
            [(TIdentifier, "target")], 
            []),
    AstNode("CGoto", 
            [], 
            []),
    AstNode("CNull", 
            [], 
            []),
    AstNode("Int", 
            [], 
            []),
    AstNode("CStatic", 
            [], 
            []),
    AstNode("CFunctionDeclaration",
           [(TIdentifier, "target"), (TIdentifier, "[params")],
           ["body", "fun_type", "storage_class"]),
]

In [7]:
def print_func():
    print("static void print_ast(Ast* node, size_t t) {")
    print("    if(!node) {")
    print("         field(\"None\", \"\", ++t);")
    print("         return;")
    print("     }")
    print("     switch(node->type()) {")
    for node in ast_nodes:
        print_node(node)
#     for node in ast_bases:
#         print_base(node)
    print("        default:")
    print("            raise_runtime_error(\"Ast pretty print not implemented for node type \" + em(std::to_string(node->type())));")
    print("    }")
    print("}")

In [8]:
print_func()

static void print_ast(Ast* node, size_t t) {
    if(!node) {
         field("None", "", ++t);
         return;
     }
     switch(node->type()) {
        case AST_T::CAdd_t: {
            field("CAdd", "", ++t);
            CAdd* p_node = static_cast<CAdd*>(node);
            break;
        }
        case AST_T::CConstInt_t: {
            field("CConstInt", "", ++t);
            CConstInt* p_node = static_cast<CConstInt*>(node);
            field("TInt", std::to_string(p_node->value), t+1);
            break;
        }
        case AST_T::CConstLong_t: {
            field("CConstLong", "", ++t);
            CConstLong* p_node = static_cast<CConstLong*>(node);
            field("TLong", std::to_string(p_node->value), t+1);
            break;
        }
        case AST_T::CConstant_t: {
            field("CConstant", "", ++t);
            CConstant* p_node = static_cast<CConstant*>(node);
            print_ast(p_node->constant.get(), t);
            break;
        }
        case AST_T::C